# Extract data from GEOTIFF files

In [ ]:
import os
import numpy as np
import pandas as pd
import rasterio
from datetime import datetime
from tqdm import tqdm

def parse_timestamp_from_filename_hima(file_name):
    """Extract the timestamp with date and hour from the Hima filename."""
    parts = file_name.split('_')
    timing = parts[1]
    subparts = timing.split('.Z')

    date_str = subparts[0]  # YYYYmmdd
    hour_str = subparts[1]  # HHMM in ZHHMM format
    
    timestamp = datetime.strptime(date_str + hour_str, '%Y%m%d%H%M')
    return timestamp

def parse_timestamp_from_filename_era5_aws(file_name):
    """Extract timestamp from ERA5 and AWS filenames."""
    parts = file_name.split('_')
    date_str = parts[1].split('.')[0]  # YYYYmmddHHMMSS format
    timestamp = datetime.strptime(date_str, '%Y%m%d%H%M%S')
    return timestamp

def extract_image_data(file_path):
    """Extract the 90x250 pixel array from a .tif file."""
    try:
        with rasterio.open(file_path) as src:
            data = src.read(1)  # Read the first band
        return data
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return np.full((90, 250), np.nan)  # NaN for missing data

def extract_band_or_variable_name(file_path, root_dir):
    """
    Extract the <Band> or <Variable> name from the directory structure:
    DATA_SV/Hima/<Band>/<Year>/<Month>/<Day>/<File>
    """
    # Split the file path relative to the root directory
    relative_path = os.path.relpath(file_path, root_dir)
    parts = relative_path.split(os.sep)  # Split by folder separators

    # The band/variable name will always be the first part after the root directory
    # print(parts[0])
    return parts[0]  # <Band> or <Variable>

def index_files_by_timestamp_and_feature(root_dir, parse_fn):
    """
    Index files by their timestamp and feature name for fast lookup.
    Example: {timestamp: {'B04B': 'path/to/B04B_YYYYMMDDHHMM.tif', ...}}
    """
    file_dict = {}
    for root, _, files in os.walk(root_dir):
        for file in files:
            if file.endswith('.tif'):
                timestamp = parse_fn(file)
                feature_name = extract_band_or_variable_name(root, root_dir)  # Extract band/variable

                if timestamp not in file_dict:
                    file_dict[timestamp] = {}
                file_dict[timestamp][feature_name] = os.path.join(root, file)

    return file_dict

def process_data(hima_dir, era5_dir, aws_dir):
    """Process all files and extract features and labels."""
    dataset = []

    # Index Hima and ERA5 files by timestamp and feature
    print("Indexing Hima files...")
    hima_files = index_files_by_timestamp_and_feature(hima_dir, parse_timestamp_from_filename_hima)

    print("Indexing ERA5 files...")
    era5_files = index_files_by_timestamp_and_feature(era5_dir, parse_timestamp_from_filename_era5_aws)

    # Collect all possible feature names across all timestamps
    all_hima_features = set()  # Unique Hima band names
    all_era5_features = set()  # Unique ERA5 variable names

    for files in hima_files.values():
        all_hima_features.update(files.keys())
    for files in era5_files.values():
        all_era5_features.update(files.keys())

    # Ensure a consistent list of feature names across all rows
    all_features = list(all_hima_features) + list(all_era5_features)

    # Traverse AWS files and extract labels and features
    aws_files = [
        os.path.join(root, file)
        for root, _, files in os.walk(aws_dir)
        for file in files if file.endswith('.tif')
    ]

    for aws_path in tqdm(aws_files, desc="Processing AWS files"):
        aws_data = extract_image_data(aws_path)
        aws_timestamp = parse_timestamp_from_filename_era5_aws(os.path.basename(aws_path))

        # Find rows and cols where AWS stations exist (non-zero values)
        rows, cols = np.where(aws_data > 0)  # Station locations

        for row, col in zip(rows, cols):
            aws_value = aws_data[row, col]

            # Extract Himawari features for this timestamp and location
            hima_features = {name: np.nan for name in all_hima_features}  # Default NaN
            if aws_timestamp in hima_files:
                for band_name, band_path in hima_files[aws_timestamp].items():
                    hima_data = extract_image_data(band_path)
                    hima_features[band_name] = hima_data[row, col]

            # Extract ERA5 features for this timestamp and location
            era5_features = {name: np.nan for name in all_era5_features}  # Default NaN
            if aws_timestamp in era5_files:
                for var_name, var_path in era5_files[aws_timestamp].items():
                    era5_data = extract_image_data(var_path)
                    era5_features[var_name] = era5_data[row, col]

            # Store the data as a tuple (timestamp, row, col, aws_value, all features)
            feature_tuple = (
                aws_timestamp, row, col, aws_value,
                *hima_features.values(),
                *era5_features.values()
            )
            dataset.append(feature_tuple)

    # Create column names dynamically
    feature_names = ['timestamp', 'row', 'col', 'aws_value'] + all_features

    # Convert dataset to DataFrame
    df = pd.DataFrame(dataset, columns=feature_names)

    return df

# Example usage
hima_dir = 'DATA_SV_ver2/DATA_SV/Hima'
era5_dir = 'DATA_SV_ver2/DATA_SV/ERA5'
aws_dir = 'DATA_SV_ver2/DATA_SV/Precipitation/AWS'
# output_csv = 'aligned_data.csv'

df = process_data(hima_dir, era5_dir, aws_dir)
print(df.head())

# Save to CSV for further analysis
df.to_csv('rainfall_dataset_1.csv', index=False)

Indexing Hima files...
Indexing ERA5 files...


Processing AWS files: 100%|██████████| 2807/2807 [1:19:47<00:00,  1.71s/it]


   timestamp  row  col  aws_value        B09B       VSB        B16B      B06B  \
0 2019-04-01    3  115        0.2  252.925888  0.092611  268.228333  0.038708   
1 2019-04-01    5  114        0.8  252.382706  0.063253  266.749786  0.030103   
2 2019-04-01    6  116        0.6  250.753403  0.094564  261.886749  0.055151   
3 2019-04-01    7  115       19.0  250.763870  0.198283  260.937897  0.059070   
4 2019-04-01   15  115        1.2  250.872086  0.147400  265.053253  0.057507   

         B14B         I4B  ...      U850      V250       R250        EWSS  \
0  282.099701  283.889587  ... -3.606674  4.523453  33.599831 -641.113281   
1  280.077759  281.936829  ... -3.606674  4.523453  33.599831 -641.113281   
2  272.486755  277.685822  ... -3.260971  3.843765  20.486549 -171.113281   
3  270.996948  275.486755  ... -5.014877  3.968765  21.638893 -585.113281   
4  278.369141  283.916656  ... -5.737534  4.207047  13.377175 -354.113281   

      SSHF      TCLW       U250       TCWV        

# Handle missing data

In [ ]:
import pandas as pd
df = pd.read_csv('rainfall_dataset.csv')
for col in df.columns:
    print(df[col].isnull().value_counts())  

timestamp
False    85757
Name: count, dtype: int64
row
False    85757
Name: count, dtype: int64
col
False    85757
Name: count, dtype: int64
aws_value
False    85757
Name: count, dtype: int64
B09B
False    81866
True      3891
Name: count, dtype: int64
VSB
True     44399
False    41358
Name: count, dtype: int64
B16B
False    81864
True      3893
Name: count, dtype: int64
B06B
True     49449
False    36308
Name: count, dtype: int64
B14B
False    81864
True      3893
Name: count, dtype: int64
I4B
False    80795
True      4962
Name: count, dtype: int64
I2B
False    81864
True      3893
Name: count, dtype: int64
B05B
True     45647
False    40110
Name: count, dtype: int64
B10B
False    81866
True      3891
Name: count, dtype: int64
WVB
False    81862
True      3895
Name: count, dtype: int64
B11B
False    81866
True      3891
Name: count, dtype: int64
IRB
False    81864
True      3893
Name: count, dtype: int64
B04B
True     44493
False    41264
Name: count, dtype: int64
B12B
False    81866


In [8]:
import pandas as pd

# Load the processed dataset
df = pd.read_csv('rainfall_dataset.csv')

# Identify the protected features
protected_features = ['timestamp', 'row', 'col', 'aws_value']

# 1. Remove all rows with any null value (including non-protected features)
df_no_nulls = df.dropna()
print(f"Dataset 1 - No Nulls: {df_no_nulls.shape}")

# 2. Remove specified features and fill missing values with mean (excluding protected features)
features_to_remove = ['VSB', 'B06B', 'B05B', 'B04B']
df_reduced = df.drop(columns=features_to_remove, errors='ignore')

# Separate protected and non-protected features
non_protected_features = df_reduced.columns.difference(protected_features)
df_non_protected = df_reduced[non_protected_features]

# Fill missing values in non-protected features with their column-wise mean
df_non_protected_filled = df_non_protected.fillna(df_non_protected.mean())

# Combine the protected features with the filled non-protected features
df_filled = pd.concat([df_reduced[protected_features], df_non_protected_filled], axis=1)
print(f"Dataset 2 - Filled Missing Values: {df_filled.shape}")

# 3. Remove specified features and drop rows with any remaining nulls (excluding protected features)
df_reduced_no_nulls = df_reduced.dropna(subset=non_protected_features)
print(f"Dataset 3 - Reduced Features with No Nulls: {df_reduced_no_nulls.shape}")

# Save the datasets to CSV files
df_no_nulls.to_csv('rainfall_dataset_no_nulls.csv', index=False)
df_filled.to_csv('rainfall_dataset_filled.csv', index=False)
df_reduced_no_nulls.to_csv('rainfall_dataset_reduced_no_nulls.csv', index=False)

print("Datasets saved successfully.")


Dataset 1 - No Nulls: (34778, 38)
Dataset 2 - Filled Missing Values: (85757, 34)
Dataset 3 - Reduced Features with No Nulls: (80792, 34)
Datasets saved successfully.


# Handle outliers & Normalize data

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Load the dataset
df_no_nulls = pd.read_csv('datasets/rainfall_dataset_reduced_no_nulls.csv')

# Exclude 'row' and 'col' from outlier detection and normalization
features_to_process = [col for col in df_no_nulls.columns if col not in ['timestamp', 'row', 'col', 'aws_value']]

# Function to detect and remove outliers using the IQR method
def remove_outliers_iqr(df, columns):
    for column in columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        # Remove outliers
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# Apply outlier removal
df_cleaned = remove_outliers_iqr(df_no_nulls, features_to_process)

# Normalize features excluding 'row' and 'col'
scaler = MinMaxScaler()
df_cleaned[features_to_process] = scaler.fit_transform(df_cleaned[features_to_process])

# Export the cleaned and normalized dataset to CSV
df_cleaned.to_csv('datasets/cleaned_normalized_rainfall_data.csv', index=False)

print("Outliers removed, data normalized, and exported to 'cleaned_normalized_rainfall_data.csv'.")


Outliers removed, data normalized, and exported to 'cleaned_normalized_rainfall_data.csv'.
